In [4]:
#libraries
import os
import pandas as pd
from datetime import datetime as dt
from datetime import time as t
import numpy as np
from IPython.display import display

In [5]:
## PLEASE INPUT YOUR OWN CREDENTIAL BELOW
user_credential = r'/Users/dinhhoang.nguyen.CONCENTRIX/OneDrive - Concentrix Corporation\WFM-internal'

In [6]:
# IMPORT MASTER ROSTER
masterroster = pd.read_json(os.path.join(r'C:', user_credential, r'DB\filejson\masterroster.json'))
masterroster['Employee_ID'] = masterroster['Employee_ID'].astype("Int64")

In [7]:
# IMPORT EPS
eps = pd.read_json(os.path.join(r'C:', user_credential, r'DB\filejson\eps.json'))


In [8]:
#IMPORT CPI
cpi = pd.read_json(os.path.join(r'C:', user_credential, r'DB\filejson\cpi.json'))

In [9]:
#IMPORT AHT
aht = pd.read_json(os.path.join(r'C:', user_credential, r'DB\filejson\aht.json'))

In [10]:
# IMPORT SCHEDULE
schedule = pd.read_json(os.path.join(r'C:', user_credential, r'DB\filejson\schedule.json'))

In [11]:
# IMPORT CUIC
cuic = pd.read_json(os.path.join(r'C:', user_credential, r'DB\filejson\cuic.json'))

In [12]:
# IMPORT TL
tl = pd.read_json(os.path.join(r'C:', user_credential, r'DB\filejson\tl.json'))

In [13]:
# Frame
aht_frame = schedule

## add Week
aht_frame['Date'] = pd.to_datetime(aht_frame['Date'], errors='coerce')
aht_frame['Week'] = aht_frame['Date'].dt.strftime('%Y%W')
aht_frame['Date'] = aht_frame['Date'].dt.date

## add tenure
aht_frame = pd.merge(aht_frame, masterroster[['Employee_ID', 'Booking Login ID', 'TED Name', 'CUIC Name', 'PST_Start_Date', 'Wave #', 'Role']], left_on='Emp ID', right_on='Employee_ID', how='left')
aht_frame['PST_Start_Date'] = pd.to_datetime(aht_frame['PST_Start_Date'], format='mixed').dt.date
def TN(x):
    if x['Date'] - x['PST_Start_Date'] >= pd.Timedelta(days=90):
        return "TN"
    else:
        return "NH"
    
aht_frame['Tenure'] = aht_frame.apply(TN, axis=1)
aht_frame = aht_frame.drop(columns=['Employee_ID'])
## add supervisor
aht_frame = pd.merge(aht_frame, tl[['Emp ID', 'Supervisor']], on='Emp ID', how='left')

In [14]:
## cpi processing
aht_cpi = cpi

## add phone cases
aht_cpi['#phone'] = aht_cpi.apply(lambda x: x['Number of Records'] if x['Channel'] == 'phone' else 0, axis=1)
aht_cpi['#non-phone'] = aht_cpi.apply(lambda x: x['Number of Records'] if x['Channel'] != 'phone' else 0, axis=1)

aht_cpi = aht_cpi.groupby(['Employee_ID', 'Date'], as_index=False)[['#phone', '#non-phone']].sum()
aht_cpi['total_cases'] = aht_cpi['#phone'] + aht_cpi['#non-phone']
### add to main frame
aht_cpi['Date'] = aht_cpi['Date'].dt.date
aht_frame = pd.merge(aht_frame, aht_cpi, left_on=['Emp ID', 'Date'], right_on=['Employee_ID', 'Date'], how='left')
aht_frame = aht_frame.drop(columns={'Employee_ID'})
aht_frame[['#phone', '#non-phone', 'total_cases']] = aht_frame[['#phone', '#non-phone', 'total_cases']].fillna(0)



In [15]:
## AHT
aht_aht = aht.drop(columns=['Start Time', 'End Time', '"Handling Time"', 'First Email Id'])

## add Eliminate Phone
def eliminate_phone(x):
    if x['Item Channel'] =='Phone' and x['Topic'] =='Unknown':
        return "Notphone"
    else:
        return x['Item Channel']
aht_aht['Eliminate Phone'] = aht_aht.apply(eliminate_phone, axis=1)

aht_mr = masterroster[['Employee_ID', 'PST_Start_Date', 'CUIC Name', 'Full name', 'Role', 'TED Name', 'Wave #']]

aht_aht = pd.merge(aht_aht, aht_mr, left_on='Staff', right_on='TED Name', how='left')

aht_aht = pd.merge(aht_aht, tl, left_on ='Employee_ID_x', right_on = 'Emp ID', how='left')
aht_lob = schedule[['Emp ID', 'Date', 'LOB']]
aht_lob['Date'] = pd.to_datetime(aht_lob['Date'], format ='mixed')
aht_aht = pd.merge(aht_aht, aht_lob, left_on = ['Employee_ID_x', 'Date'], right_on=['Emp ID', 'Date'], how='left')
aht_aht = aht_aht.drop(columns=['Employee_ID_y', 'Emp ID_x', 'Emp ID_y'])
aht_aht = aht_aht.rename(columns={'Employee_ID_x':'Employee_ID'})
#Add talk time
def talk_time(x):
    try:
        from_left = 11
        to_right = str(x['Tooltip Phone Time']).find(' Wrap Time')
        return int(x['Tooltip Phone Time'][from_left:to_right])
    except:
        return 0
def wrap_time(x):
    try:
        from_left = int(str(x['Tooltip Phone Time']).find('Wrap Time'))+11
        to_right = str(x['Tooltip Phone Time']).find(' Hold Time')
        return int(x['Tooltip Phone Time'][from_left:to_right])
    except:
        return 0
def hold_time(x):
    try:
        from_left = int(str(x['Tooltip Phone Time']).find('Hold Time'))+10
        return int(x['Tooltip Phone Time'][from_left:])
    except:
        return 0    
aht_aht['Talk Time'] = aht_aht.apply(talk_time, axis=1)
aht_aht['Wrap Time'] = aht_aht.apply(wrap_time, axis=1)
aht_aht['Hold Time'] = aht_aht.apply(hold_time, axis=1)

def act_handling_time(x):
    if x['Eliminate Phone'] =='Phone':
        return x['Talk Time'] + x['Wrap Time'] + x['Hold Time']
    else:
        return x['Handling Time']

# add realchannel
def realchannel(x):
    if x['Item Channel'] == 'Phone' and x['Topic'] == 'Unknown':
        return "Phone1"
    else:
        return x['Item Channel']
aht_aht['Real Channel'] = aht_aht.apply(realchannel, axis=1)

# add total inbound
def totalinbound(x):
    if x['Item Channel'] in ['Email', 'Live Chat', 'Messaging', 'Unknown']:
        return x['Handling Time']
    else:
        return x['Talk Time'] + x['Wrap Time'] + x['Hold Time']

aht_aht['Total Inbound'] = aht_aht.apply(totalinbound, axis=1)

#add Act handling time
aht_aht['Act Handling Time'] = aht_aht.apply(act_handling_time, axis=1)
#Date to_datetime
aht_aht['Date'] = pd.to_datetime(aht_aht['Date'], format='mixed').dt.strftime('%Y-%m-%d')


C:\Users\dinhhoang.nguyen.CONCENTRIX\AppData\Local\Temp\VSCodePortable-x64Temp\ipykernel_2264\87916195.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aht_lob['Date'] = pd.to_datetime(aht_lob['Date'], format ='mixed')


In [16]:
# time & count for AHT
aht_aht['AHT Phone time'] = aht_aht.apply(lambda x: x['Act Handling Time'] if x['Eliminate Phone'] == 'Phone' else 0, axis=1)
aht_aht['AHT Phone count'] = aht_aht.apply(lambda x: 1 if x['Eliminate Phone'] == 'Phone' else 0, axis=1)

aht_aht['AHT Non-phone time'] = aht_aht.apply(lambda x: x['Act Handling Time'] if x['Eliminate Phone'] in ['Email', 'Unknown', 'Messaging', 'SNR', 'BH'] else 0, axis=1)
aht_aht['AHT Non-phone count'] = aht_aht.apply(lambda x: 1 if x['Eliminate Phone'] in ['Email', 'Unknown', 'Messaging', 'SNR', 'BH'] else 0, axis=1)

aht_aht['Overall AHT time'] = aht_aht.apply(lambda x: x['Act Handling Time'] if x['Eliminate Phone'] != 'Notphone' else 0, axis=1)
aht_aht['Overall AHT count'] = aht_aht.apply(lambda x: 1 if x['Eliminate Phone'] != 'Notphone' else 0, axis=1)

aht_aht['Hold (phone) time'] = aht_aht.apply(lambda x: x['Hold Time'] if x['Eliminate Phone'] == 'Phone' else 0, axis=1)
aht_aht['Hold (phone) count'] = aht_aht.apply(lambda x: 1 if x['Eliminate Phone'] == 'Phone' else 0, axis=1)

aht_aht['AACW (phone) time'] = aht_aht.apply(lambda x: x['Wrap Time'] if x['Eliminate Phone'] == 'Phone' else 0, axis=1)
aht_aht['AACW (phone) count'] = aht_aht.apply(lambda x: 1 if x['Eliminate Phone'] == 'Phone' else 0, axis=1)

aht_aht['Avg Talk Time time'] = aht_aht.apply(lambda x: x['Talk Time'] if x['Eliminate Phone'] == 'Phone' else 0, axis=1)
aht_aht['Avg Talk Time count'] = aht_aht.apply(lambda x: 1 if x['Eliminate Phone'] == 'Phone' else 0, axis=1)

aht_aht['Handled call time'] = aht_aht.apply(lambda x: x['Total Inbound'] if x['Real Channel'] == 'Phone' else 0, axis=1)
aht_aht['Handled call count'] = aht_aht.apply(lambda x: 1 if x['Real Channel'] == 'Phone' else 0, axis=1)

aht_aht['Handled email time'] = aht_aht.apply(lambda x: x['Total Inbound'] if x['Real Channel'] in ['Email', 'Live Chat', 'Messaging'] else 0, axis=1)
aht_aht['Handled email count'] = aht_aht.apply(lambda x: 1 if x['Real Channel'] in ['Email', 'Live Chat', 'Messaging'] else 0, axis=1)

In [ ]:
# group by AHT
aht_aht = aht_aht.groupby(['Date', 'Employee_ID'], as_index=False)[['AHT Phone time', 'AHT Phone count', 'AHT Non-phone time', 'AHT Non-phone count', 'Overall AHT time', 'Overall AHT count', 'Hold (phone) time', 'Hold (phone) count', 'AACW (phone) time', 'AACW (phone) count', 'Avg Talk Time time', 'Avg Talk Time count', 'Handled call time', 'Handled call count', 'Handled email time', 'Handled email count']].sum()

In [ ]:
# add aht_aht to main frame
aht_frame['Date'] = pd.to_datetime(aht_frame['Date'], format='mixed').dt.strftime('%Y-%m-%d')
aht_frame = pd.merge(aht_frame, aht_aht, left_on =['Emp ID', 'Date'], right_on=['Employee_ID', 'Date'], how='left')
aht_frame = aht_frame.drop(columns={'Employee_ID'})
aht_frame['AHT Phone time'] = aht_frame['AHT Phone time'].fillna(0)
aht_frame['AHT Phone count'] = aht_frame['AHT Phone count'].fillna(0)

aht_frame['AHT Non-phone time'] = aht_frame['AHT Non-phone time'].fillna(0)
aht_frame['AHT Non-phone count'] = aht_frame['AHT Non-phone count'].fillna(0)

aht_frame['Overall AHT time'] = aht_frame['Overall AHT time'].fillna(0)
aht_frame['Overall AHT count'] = aht_frame['Overall AHT count'].fillna(0)

aht_frame['Hold (phone) time'] = aht_frame['Hold (phone) time'].fillna(0)
aht_frame['Hold (phone) count'] = aht_frame['Hold (phone) count'].fillna(0)

aht_frame['AACW (phone) time'] = aht_frame['AACW (phone) time'].fillna(0)
aht_frame['AACW (phone) count'] = aht_frame['AACW (phone) count'].fillna(0)

aht_frame['Avg Talk Time time'] = aht_frame['Avg Talk Time time'].fillna(0)
aht_frame['Avg Talk Time count'] = aht_frame['Avg Talk Time count'].fillna(0)

aht_frame['Handled call time'] = aht_frame['Handled call time'].fillna(0)
aht_frame['Handled call count'] = aht_frame['Handled call count'].fillna(0)

aht_frame['Handled email time'] = aht_frame['Handled email time'].fillna(0)
aht_frame['Handled email count'] = aht_frame['Handled email count'].fillna(0)

In [ ]:
# cuic
aht_cuic = cuic
aht_cuic = aht_cuic.groupby(['Employee_ID', 'Session Date'], as_index=False)[['AgentAvailTime']].sum()
aht_cuic['AgentAvailTime']=aht_cuic['AgentAvailTime']*24
aht_cuic['Session Date'] = pd.to_datetime(aht_cuic['Session Date'], format='mixed').dt.strftime('%Y-%m-%d')

In [ ]:
# add cuic to main frame
aht_frame = pd.merge(aht_frame, aht_cuic, left_on =['Emp ID', 'Date'], right_on=['Employee_ID', 'Session Date'], how='left')
aht_frame = aht_frame.drop(columns={'Employee_ID', 'Session Date'})
aht_frame['AgentAvailTime'] = aht_frame['AgentAvailTime'].fillna(0)

In [ ]:
# eps
aht_eps = eps
# calculate time
aht_eps['phone_time'] = aht_eps.apply(lambda x: x['Total Time'] if x['BPE Code'] == 'Ready or Talking' else 0, axis=1)/3600
aht_eps['nonphone_time'] = aht_eps.apply(lambda x: x['Total Time'] if x['BPE Code'] == 'Picklist - off Phone' else 0, axis=1)/3600
aht_eps['productive_time'] = aht_eps.apply(lambda x: x['Total Time'] if x['BPE Code'] in ['Picklist - off Phone', 'Ready or Talking'] else 0, axis=1)/3600

## add session date
aht_eps['Session login date'] = pd.to_datetime(aht_eps['Session login date'], format='mixed').dt.date
aht_eps['Session login date-1'] = aht_eps['Session login date'] - pd.Timedelta(days=1)
aht_eps = pd.merge(aht_eps, schedule[['Emp ID', 'Date', 'Shift_type']], left_on=['EID', 'Session login date-1'], right_on=['Emp ID', 'Date'], how='left')

def session_date(x):
    if x['Shift_type'] == 'NS' and x['Session login time'] <= t(12, 0, 0):
        return x['Session login date-1']
    else:
        return x['Session login date']
    
aht_eps['Session login time'] = pd.to_datetime(aht_eps['Session login time'], format='mixed').dt.time
aht_eps['Session date'] = aht_eps.apply(session_date, axis=1)

aht_eps = aht_eps.groupby(['EID', 'Session date'], as_index=False)[['phone_time', 'nonphone_time', 'productive_time']].sum()
aht_eps = aht_eps.rename(columns={'Session date': 'Date'})

aht_eps['Date'] = pd.to_datetime(aht_eps['Date'], format = 'mixed').dt.strftime('%Y-%m-%d')

In [ ]:
## add aht_eps to main frame
aht_frame = pd.merge(aht_frame, aht_eps, left_on =['Emp ID', 'Date'], right_on=['EID', 'Date'], how='left')
aht_frame = aht_frame.drop(columns={'EID'})
aht_frame[['phone_time', 'nonphone_time', 'productive_time']] = aht_frame[['phone_time', 'nonphone_time', 'productive_time']].fillna(0)


In [ ]:
aht_frame['Date'] = pd.to_datetime(aht_frame['Date'], format='mixed').dt.strftime('%Y-%m-%d')

In [ ]:
# export to json
aht_frame.to_json(os.path.join(r'C:', user_credential, r'DB\filejson\ahtrp.json'), orient='records')